# Week 4 - Second Year Project

---

**Learning goals**
* Be able to inspect what is represented in word embeddings
* Understand and implement cosine distance to compare two representations
* Understand and implement the CBOW model in word2vec
* Be able to prepare data for input into a neural language model
* Read and understand an academic paper
* Understand how filters and channels are used in a CNN for NLP

**Notebook overview**

*Lecture 7: What do word embeddings represent?*

1. Word similarities
2. Analogies
3. Representing the data
4. Implement the continuous bag of words model for estimating word embeddings
5. Save & inspect learned word embeddings

*Lecture 8: CNN*

6. Convolutional Neural Networks for Sentence Classification

# Lecture 7: What do word embeddings represent?
In this exercise, you are going to explore what is represented with word embeddings. You are going to make use of the python gensim package and two sets of pre-trained embeddings. The embeddings can be downloaded from:

* http://itu.dk/people/robv/data/embeds/twitter.bin.tar.gz
* http://itu.dk/people/robv/data/embeds/GoogleNews-50k.bin.tar.gz

The first embeddings are skip-gram embeddings trained on a collection of 2 billion words from English tweets collected during 2012 and 2018 with the default settings of word2vec. The second embeddings are trained on 100 billion words from Google News. They have both been truncated to the most frequent 500,000 words. Note that loading that each of these embeddings require approximately 2GB of ram.

The embeddings can be loaded in gensim as follows:

In [ ]:
import gensim.models

twitEmbs = gensim.models.KeyedVectors.load_word2vec_format(
                                'twitter.bin', binary=True)
print('loading finished')

You can now use the index operator ``[]`` or the function ``get_vector()`` to acces the individual word embeddings.

In [ ]:
twitEmbs['cat']

## 1. Word similarities
Cosine distance can be used to measure the distance between two words. It is defined as:
\begin{equation}
cos_{\vec{a},\vec{b}} = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}| |\vec{b}|} = \frac{\sum^n_1 a_i b_i}{\sqrt{\sum^n_1 a_i^2} \sqrt{\sum^n_1 b_i^2}}
\end{equation}

* a) Implement the cosine similarity using pure python (only the ``math`` package is allowed). Note that `similarity == 1-distance`.

You can compare your scores to the gensim implementation to check wheter it is correct. The following code should give the same output

```
print(twitEmbs.distance('cat', 'dog'))
print(cosine(twitEmbs['cat'], twitEmbs['dog']))
```


In wordnet, the distance between two senses can be based on the distance in the taxonomy. The most common metric for this is:

* Wu-Palmer Similarity: denotes how similar two word senses are, based on the depth of the two senses in the taxonomy and of their Least Common Subsumer (most specific ancestor node).

It can be obtained in python like this:

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet

first_word = wordnet.synsets('cat')[0] #0 means: most common sense
second_word = wordnet.synsets('dog')[0]
print('WordNet similarity: ' + str(first_word.wup_similarity(second_word)))

print('Twitter similarity: ' + str(twitEmbs.similarity('cat', 'dog')))




* b) Think of 5 word pairs which have a high similarity according to you. Estimate the difference between these pairs in wordnet as well as in the Twitter embeddings and the Google News embeddings. Which method is closest to your own intuition? (You are allowed to use the gensim implementation of cosine similarity here.)


## 2. Analogies

Analogies have often been used to demonstrate the power of word embeddings. Analogies have the form ``A :: B : C :: D``. In this setting `A`, `B` and `C` are usually given and the fourth term `D` is extracted from the embeddings by using ``3cosadd``:

\begin{equation}
\underset{d}{\mathrm{argmax}} (\cos (d, c) - \cos (d, a) + \cos (d, b))
\label{equ:cosadd}
\end{equation}

You can query analogies with gensim:

In [ ]:
twitEmbs.most_similar(positive=['woman', 'king'], negative=['man'], 
                                                         topn=10)

``3cosadd`` can be used to solve semantic as well as syntactic analogies:

| Semantic            |                                      |
|---------------------|--------------------------------------|
| Country-capital     | Denmark :: Copenhagen : England :: X |
| Family-relations    | boy :: girl : he :: X                |
| Object-color        | sky :: blue : grass :: X             |

| Syntactic           |                                      |
|---------------------|--------------------------------------|
| Superlatives        | nice :: nicer : good :: X            |
| Present-past tense  | work :: worked : drink :: X          |
| Country-nationality | Brazil :: Brazilian : Denmark :: X   |


Try the analogies from the table. Is the correct answer returned for all queries? 
If not: are the answers at least ranked high?

* a) Think of another category of *semantic* analogies that might be encoded in the embeddings and test this empirically by thinking of 5 example analogies. Which embeddings are better at predicting your category (Twitter versus Google News)?

* b) Think of another category of *syntactic* analogies that might be encoded in the embeddings and test this empirically by thinking of 5 example analogies. Which embeddings are better at predicting your category (Twitter versus Google News)?


# Learning Word Embeddings


So far you've learned about distributional semantics (vector semantics) in both the traditional and modern neural way, and you qualitatively worked with pre-trained (off-the-shelf) word embeddings in the last assignment.

In this assignment, you will learn how to implement a neural network  to learn word embeddings, namely the *Continous Bag of Words* (CBOW) model for word embeddings. More specifically, you will:

* learn how to represent text for windows language modeling
* learn how to design a Pytorch model (`torch.module`)
* learn how to implement a FNN for learning embeddings with CBOW which *sums* the context embedding vectors
* train the model for a few epochs using stochastic gradient descent (SGD)
* read off the learned embeddings $W$, store them in a gensim-readable file and inspect them


### CBOW



CBOW is a model proposed by [Mikolov et al., 2013](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf).

It is a simple neural method to learn word embeddings and it is one of the two core algorithms in the `word2vec` toolkit (see figure below). Note that, besides its usage here to learn word embeddings, CBOW is also a more general term used to refer to any input representation which consists of (some) way of aggregating a set of word embeddings. Hence its name, the continous BOW representation. You can in fact use such a similar representation (e.g., the average of the embeddings of words) for other tasks as well, such as text classification. Here, CBOW is meant in its original formulation: a network over the *sum* of embeddings of context words aimed at predicting the middle target word. It is related in spirit to a language model, but instead framed as a classification task (with context available on both sides) and hence bears more similarities with a *[word close test](https://en.wikipedia.org/wiki/Cloze_test)*.

Illustration of the CBOW model (in comparison to the skip-gram):
<img src="pics/cbow-vs-skipgram.png">

##  3. Representing the data

Given a corpus, extract the training data for the CBOW model using a window size of 2 words on each side of the target word. The following image shows what the input of the training algorithm (`Input`) should look like (`Training window`):


<img src="pics/cbow-window.jpg">

Hints:
* Remember to `"<pad>"` the input when the window size is smaller than the expected window size. This also means that the `"<pad>"` token should be in the vocabulary; reserve the first `0` index for this special token.
* In Pytorch, all input is expected to be a `torch.tensor`. You can create these beforehand with `torch.zeros()`, or just convert a resulting python list by using `torch.tensor(train_data)`.

Example:

Given the following tiny corpus:
```
tiny_corpus = ["this is an example", "this is a longer example sentence", "I love deep learning"]
```

To create the `train_X` data, you first need to extract n-gram windows and the target words:

```
label,context
this ['<pad>', '<pad>', 'is', 'an']
is ['<pad>', 'this', 'an', 'example']
example ['this', 'is', 'example', '<pad>']
...
```

And convert them into numeric format, where each word token is represented by its unique index:

```
train_labels = [ 1,  2,  3,  4,  1,  2,  5,  6,  4,  7,  8,  9, 10, 11]
train_data = [[ 0,  0,  2,  3],
 [ 0,  1,  3,  4],
 [ 1,  2,  4,  0],
 [ 2,  3,  0,  0],
 [ 0,  0,  2,  5],
 [ 0,  1,  5,  6],
 [ 1,  2,  6,  4],
 [ 2,  5,  4,  7],
 [ 5,  6,  7,  0],
 [ 6,  4,  0,  0],
 [ 0,  0,  9, 10],
 [ 0,  8, 10, 11],
 [ 8,  9, 11,  0],
 [ 9, 10,  0,  0]]
```

In [ ]:
tiny_corpus = ["this is an example", "this is a longer example sentence", "I love deep learning"]

Suggestion: Implement all your steps first on the `tiny_corpus` data. Then test your implementation on the provided data `sample.txt`.

In [ ]:
## global settings
PAD = "<PAD>"
window_size=2

### your code here


##  4. Implement the continuous bag of words model for estimating word embeddings

Implement the CBOW model for word embeddings: a CBOW with window size 2, which `sums` the input embeddings and from that hidden representation `predicts` the target token. 

The steps for CBOW are as follows:
* Convert your data to the center/window (done in previous assignment
* The model should have an embedding layer a linear layer (and optionally a loss function, you can also put the loss function in the forward loop)
* In the forward function of the model, it should: look up the embeddings, sum them, convert to logits (in the linear layer), and optionally calculate the loss (can also be done in forward loop)
* In the training loop (assignment 4b), we have a for loop for the epochs and one for the data. Within this, we call the forward function and obtain the loss after which the backward pass can be called


To train a model in Pytorch, one has to define a sub-class of `torch.nn.module` (see also [assignment3](https://github.itu.dk/robv/intro-nlp2023/blob/main/assignments/week3/train.py)). The constructor `__init__()` and the `forward()` function can then be defined to specify the structure of the network. In the `__init__` function, the layers are specified and initialized, whereas the `forward` function defines how the layers interact during a forward-pass. You can use [`torch.nn.Embedding`](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) for the embedding layer, [`torch.nn.Linear`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) for the hidden layer, and [`torch.nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) as loss function. 

For some examples we refer to this [tutorial](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py) and this [introduction](https://towardsdatascience.com/an-easy-introduction-to-pytorch-for-neural-networks-3ea08516bff2).

* a) Implement the CBOW network as described above:

**Hint**: you can print the structure of the model by simply printing the initialized variable. Make sure all the layers are represented in the forward pass.

In [ ]:
import torch
import torch.nn as nn
embed_dim = 64

class CBOW(nn.Module):
    def __init__(self, emb_dim, vocab_dim):
        super(CBOW, self).__init__()
        pass

    
    def forward(self, inputs, gold):
        return inputs

cbow_model = CBOW(embed_dim,len(word2idx))
print(cbow)

* b) Now implement the training procedure with gradient descent (`learning rate=0.001`). Go through the dataset `10` times, and update the weights after each line (`batch size = 1`). An example of a training procedure can be found on: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#train-the-network

**Hint**: you have to convert the lists created in assignment 3 to be able to do the forward pass. The forward pass expects its input to be in tensors. So for the gold labels this means we have to ensure that we do not pass a zero-dimension tensor which looks like: `tensor(1)`, but convert this to `tensor([1])`. Similarly for the training data, we convert `tensor([0, 0, 2, 3])` to `tensor([[0], [0], [2], [3]])`. This can be done with [tensor views](https://pytorch.org/docs/stable/generated/torch.Tensor.view.html#torch.Tensor.view).


##  5: Save & inspect the learned word embeddings


Now that you have trained the model:
    
* a) Extract the parameters $W$ (from the Embeddings layer), and save them in a *word embeddings file* `embeds.txt` (plain text file):
  - The first line defines the size of the embeddings: `num_words emb_size` (the number of words in the vocabularly and the defined embeddings size).
  - Every line that follows contains the word followed by the numbers of each word embedding.
* b) Load the vectors in gensim and inspect a few examples of closest words (note that the training size and time was minimal in this setup, and there is likely not much learned). Think of how you can improve the model. What impacts the accuracy of the model? And the speed of training? What are the possible ways to gauge the quality of such a model?

Example of embeddings learned from `tiny_corpus` (numbers can be different from yours):

```
> head -3 embeds.txt 
12 64
<pad> -0.5048033595085144 -1.591163158416748 1.8800480365753174 -0.3564389944076538 -0.24885627627372742 -0.8058652877807617 -2.6211788654327393 0.08112381398677826 -1.3476816415786743 -1.103865146636963 1.0328012704849243 0.2507503926753998 -1.041972041130066 -0.5575347542762756 0.4840431213378906 0.8821240067481995 -0.5162855982780457 0.6237239241600037 0.29837313294410706 0.6911399364471436 0.08433791249990463 1.2551549673080444 -0.1526130735874176 -0.7068966031074524 -0.22344042360782623 -1.4958363771438599 0.934463381767273 0.01866796240210533 1.3107978105545044 0.9627560973167419 0.7330653071403503 0.043239232152700424 1.8001024723052979 0.5885927081108093 -0.6166881322860718 0.49031350016593933 -1.9978910684585571 -0.8427903652191162 -1.4683231115341187 -0.5872414112091064 -0.1067340075969696 -1.306506872177124 0.20666570961475372 0.8913899660110474 -0.10755635797977448 -0.337430864572525 -0.496940940618515 -1.834689974784851 -0.9944869875907898 1.0383625030517578 -0.5455881953239441 0.8885291218757629 -0.38933804631233215 -0.5381456613540649 -0.836338222026825 -0.5508792996406555 1.4010215997695923 -1.4299581050872803 0.24947424232959747 -0.1625206470489502 -1.0024515390396118 0.759738564491272 1.0844918489456177 0.3157642185688019
this -0.9369654059410095 -1.6312119960784912 1.3569660186767578 -0.3015367388725281 -0.36712756752967834 -0.5038203597068787 -0.6787657141685486 0.21750451624393463 -0.3248804807662964 1.3521510362625122 0.2937549948692322 -0.3129131495952606 0.83836430311203 0.6834233999252319 0.7293476462364197 -1.960811734199524 -0.8496330380439758 0.5597900748252869 0.5100727081298828 0.07686382532119751 2.489269495010376 -1.262713074684143 -0.10204033553600311 -1.0957906246185303 -1.3282076120376587 0.8120277523994446 -0.666195273399353 -0.40798619389533997 -0.6084112524986267 -0.19338122010231018 0.3624468147754669 -0.32469242811203003 1.2604715824127197 1.7926188707351685 0.4650556743144989 0.40891459584236145 0.4918572008609772 0.5958905220031738 0.33692416548728943 0.42133066058158875 0.48872730135917664 -0.2832883894443512 -0.4098934531211853 0.6750694513320923 -0.47080767154693604 0.609721302986145 1.072028636932373 -0.27546846866607666 -0.7097866535186768 -1.0163779258728027 0.9050694704055786 -1.1937428712844849 1.4058232307434082 -0.9658770561218262 -2.9560890197753906 -0.9471196532249451 -2.19875168800354 -0.9149804711341858 0.1300773173570633 -0.549370527267456 -1.2480101585388184 0.7263423800468445 0.02889973111450672 1.0743536949157715
```

## Lecture 8 CNN

### 6. Convolutional Neural Networks for Sentence Classification

For this assignment, we are going to take a closer look at the ``Convolutional Neural Networks for Sentence Classification'' paper from Yoon Kim, which can be found here: https://aclanthology.org/D14-1181/

Read the paper, and then answer the following questions:

a) Which type of pooling is applied after the convolution operations?

b) How many filters (kernels) are applied to the input sentence?

c) What dimensions do(es) the filter(s) have?

d) What do these dimensions correspond to?

e) The channels of a CNN are different ways of representing the input (e.g. for an RGB image: red, green, blue). Which different views does Kim use for sentence classification?